# initialize the Modules

src : https://towardsdatascience.com/10-minutes-to-building-a-binary-image-classifier-by-applying-transfer-learning-to-mobilenet-eab5a8719525 ; https://www.tensorflow.org/api_docs/python/tf/keras/applications/mobilenet_v2/MobileNetV2 ; https://www.youtube.com/watch?v=w8Qx40tHeEM&ab_channel=SebastiaanMath%C3%B4t ; https://colab.research.google.com/drive/1XYAXOHiXNWqmKedCj1WRcJ6mGpr4aVC0?usp=sharing

In [ ]:
!pip3 install keras keras_applications keras_preprocessing Keras-Applications sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#store working directory
!pwd
!workingDir=$(pwd)

/content


In [ ]:
import keras

In [ ]:
keras.__version__

'2.8.0'

## Initialize the Machine Learning Modules Tensorflow

In [ ]:
import tensorflow as tf
import keras.applications.mobilenet_v2 as MN2
import numpy as np
from imageio import imread

In [ ]:

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import MobileNetV2 as MN2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
import numpy as np
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

### Initiate the model of MN2 or mobilenet

In [ ]:
model = Sequential()

### The Layers

In [1]:
"""
How about if we put the sandwich 0. MN2 1. Pooling MN2 2. Convolution?(for feature sharpening) 3. Dense sigmoid
"""
input_shape=(224, 224, 3)
x = tf.random.normal(input_shape)
model.add(MN2(include_top = False, weights="imagenet", input_shape=input_shape))
#224, 224, 3 depending on the original model in this case imagenet on MN2 is 224 224 3
#model.add(Flatten(input_shape=input_shape))
#model.add(tf.keras.layers.GlobalAveragePooling2D())
#model.add(tf.keras.layers.Reshape((100,100,1),input_shape=X.shape[1:]))
model.add(tf.keras.layers.Conv2D(128, (3,3), activation = "relu", input_shape=input_shape))
model.add(tf.keras.layers.MaxPooling2D(2,2))
model.add(tf.keras.layers.Conv2D(64, (3,3), activation = "relu"))
model.add(tf.keras.layers.MaxPooling2D(2,2))
model.add(tf.keras.layers.Conv2D(32, (3,3), activation = "relu"))
model.add(tf.keras.layers.MaxPooling2D(2,2))
#model.add(tf.keras.layers.GlobalAveragePooling2D())
#model.add(tf.keras.layers.Flatten())
#model.add(Dense(units = 256, activation = 'sigmoid'))
#model.add(Dense(units = 128, activation = 'relu'))
#model.add(Dense(units = 30, activation = 'softmax'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(Dense(1, activation = 'sigmoid'))
# set layers 0 to be untrainable
model.layers[0].trainable = False

NameError: ignored

In [13]:
#after adding the layers of cheese combine it by compile it
from tensorflow.keras.optimizers import RMSprop
#model.compile(optimizer=RMSprop(learning_rate=0.01), loss = 'binary_crossentropy', metrics = 'accuracy')
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = 'accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 128)     3584      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 128)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      73792     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 32)        18464     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 32)       0

# Retrofitting Training Data

> We are going to use yoga_poses dataset that is originated from the tensorflow   !wget -O yoga_poses.zip http://download.tensorflow.org/data/pose_classification/yoga_poses.zip
which includes 5 pose

1. Chair
2. Cobra
3. Dog
4. Tree
5. Warrior

In [ ]:
!pwd; apt install wget -y; rm -f yoga_poses.zip ; rm -rf train test; wget http://download.tensorflow.org/data/pose_classification/yoga_poses.zip; rm -rf trainingDataset; mkdir trainingDataset; mv yoga_poses.zip trainingDataset

In [ ]:
!pwd; ls; apt install zip -y; cd trainingDataset; unzip yoga_poses.zip

In [ ]:
!cd ${workingDir} 

### set variable for the training path

In [ ]:
#/home/albertstarfield/Documents/FileSekolah13(TE)/bangkit_error/runtime/Motionful-Project-Bangkit2022/src/MachineLearning/model/transferLearningRetrofitting_MobileNet
import os

trainingPath = './trainingDataset/train/'
validationPath = './trainingDataset/test/'

#Training Path
Train_chairPath = "./trainingDataset/train/chair"
trainChairPose = os.listdir(Train_chairPath)
print('total Chair Pose Images', len(trainChairPose))
Train_cobraPath = "./trainingDataset/train/cobra"
trainCobraPose = os.listdir(Train_cobraPath)
print('total cobra Pose Images', len(trainCobraPose))
train_dogPath = "./trainingDataset/train/dog"
trainDogPose = os.listdir(train_dogPath)
print('total dog Pose Images', len(trainDogPose))
train_treePath = "./trainingDataset/train/tree"
trainTreePose = os.listdir(train_treePath)
print('total tree Pose Images', len(trainTreePose))
train_warriorPath = "./trainingDataset/train/warrior"
trainWarriorPose = os.listdir(train_warriorPath)
print('total warrior Pose Images', len(trainWarriorPose))

#Testing or Validation Path
Validation_chairPath = "./trainingDataset/train/chair"
validationChairPose = os.listdir(Validation_chairPath)
print('total Chair Pose Images', len(validationChairPose))
Validation_cobraPath = "./trainingDataset/train/cobra"
validationCobraPose = os.listdir(Validation_cobraPath)
print('total cobra Pose Images', len(validationCobraPose))
Validation_dogPath = "./trainingDataset/train/dog"
validationDogPose = os.listdir(Validation_dogPath)
print('total dog Pose Images', len(validationDogPose))
Validation_treePath = "./trainingDataset/train/tree"
validationTreePose = os.listdir(Validation_treePath)
print('total tree Pose Images', len(validationTreePose))
Validation_warriorPath = "./trainingDataset/train/warrior"
trainWarriorPose = os.listdir(Validation_warriorPath)
print('total warrior Pose Images', len(trainWarriorPose))

### Visualizing dataset using data matplotlib

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plot
import matplotlib.image as mpimg

#view it as 5x5
Visrows = 5
Viscols = 5
picIndex = 0 #starts index from 0

In [ ]:
#view dataset images by using matplotlib
figure = plot.gcf()
figure.set_size_inches(Visrows * 5, Viscols * 5)
#add index +10 because we are going through 10 photos now
picIndex += 10
# view all classification of the pose
"""
Chair
Cobra
Dog
Tree
Warrior

#Training Path
Train_chairPath = "./trainingDataset/train/chair"
trainChairPose = os.listdir(Train_chairPath)
print('total Chair Pose Images', len(trainChairPose))
Train_cobraPath = "./trainingDataset/train/cobra"
trainCobraPose = os.listdir(Train_cobraPath)
print('total cobra Pose Images', len(trainCobraPose))
train_dogPath = "./trainingDataset/train/dog"
trainDogPose = os.listdir(train_dogPath)
print('total dog Pose Images', len(trainDogPose))
train_treePath = "./trainingDataset/train/tree"
trainTreePose = os.listdir(train_treePath)
print('total tree Pose Images', len(trainTreePose))
train_warriorPath = "./trainingDataset/train/warrior"
trainWarriorPose = os.listdir(train_warriorPath)
print('total warrior Pose Images', len(trainWarriorPose))
"""

seekChairData = [os.path.join(Train_chairPath, fname) for fname in trainChairPose[picIndex-10:picIndex]]
seekCobraData = [os.path.join(Train_cobraPath, fname) for fname in trainCobraPose[picIndex-10:picIndex]]
seekDogData = [os.path.join(train_dogPath, fname) for fname in trainDogPose[picIndex-10:picIndex]]
seekTreeData = [os.path.join(train_treePath, fname) for fname in trainTreePose[picIndex-10:picIndex]] 
seekWarriorData = [os.path.join(train_warriorPath, fname) for fname in trainWarriorPose[picIndex-10:picIndex]] 

for a, imPath in enumerate(seekChairData + seekCobraData + seekDogData + seekTreeData + seekWarriorData):
  subPlot = plot.subplot(Visrows, Viscols, a + 1)
  subPlot.axis('On') #i wanna see the axis and the gridline
  image = mpimg.imread(imPath)
  plot.imshow(image)
plot.show() 

### data resizing and classification

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator as dataGen

#lets target the scaling res
#rescaling 244 244
trainDatagen = dataGen(rescale=1/255) #1/255
validationDatagen = dataGen(rescale=1/255) #1/255
"""
Chair
Cobra
Dog
Tree
Warrior
"""
trainDataGenerator = trainDatagen.flow_from_directory(
    trainingPath,
    classes = ['chair', 'cobra', 'dog', 'tree', 'warrior'],
    target_size = (244, 244),
    batch_size = 120,
    class_mode='binary'
)

trainingValidationGenerator = validationDatagen.flow_from_directory(
    validationPath,
    classes = ['chair', 'cobra', 'dog', 'tree', 'warrior'],
    target_size = (244, 244),
    batch_size = 8,
    class_mode='binary',
    shuffle=False
)

# Train time!

In [ ]:
history = model.fit(trainDataGenerator,
steps_per_epoch=8,
epochs=16,
verbose=1,
validation_data = trainingValidationGenerator,
validation_steps=8)

# Export Model h5 and tflite

In [ ]:
model.evaluate(trainingValidationGenerator)

In [ ]:
model.save('motionful_prealpha.h5')

In [ ]:
!ls 

### converting h5 to tflite

In [ ]:
# Write the classificaiton
! echo "chair" > motionful_prealpha_classification.txt
! echo "cobra" >> motionful_prealpha_classification.txt
! echo "dog" >> motionful_prealpha_classification.txt
! echo "tree" >> motionful_prealpha_classification.txt
! echo "warrior" >> motionful_prealpha_classification.txt

In [ ]:
modelLoad = tf.keras.models.load_model('motionful_prealpha.h5')
converterModel = tf.lite.TFLiteConverter.from_keras_model(modelLoad)
tfLiteModelExport = converterModel.convert()
open("motionful_prealpha.tflite", "wb").write(tfLiteModelExport)

# Download zip archive the model

In [ ]:
import files
files.download('motionful_prealpha.tflite')
files.download('motionful_prealpha.h5')
files.download('motionful_prealpha_classification.txt')